In [1]:
import torch
import sys
sys.path.insert(0, '/home/mm22d016/Multi-Task')
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset
from multiTask_dataset import MultiTask_Dataset, MultiTask_DataAugmentation
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from copy import deepcopy
from sklearn.model_selection import KFold
from rdkit import Chem

/home/mm22d016/miniconda3/envs/TransPolymer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('../data/multiTask.csv')
data

,smiles,Eea,Egb,Egc,Ei
0,*CC(*)C,0.4343,7.6997,6.5196,6.1850
1,*CC(*)F,0.8740,8.3099,7.6225,7.6332
2,*CC(*)(F)F,1.1415,-99.0000,7.1498,8.1531
3,*C(F)C(*)(F)F,1.5240,6.8451,7.6787,8.5986
4,*CCC(F)(F)C(*)(F)F,0.4489,9.1316,8.6882,9.0178
...,...,...,...,...,...
3711,*Nc1c([2H])c([2H])c(*)c([2H])c1[2H],-99.0000,-99.0000,3.3666,-99.0000
3712,*CCCCCC[N+](C)(C)CCC[N+](*)(C)C,-99.0000,-99.0000,0.2161,-99.0000
3713,*CCCCCCCC[N+](C)(C)CCCCCC[N+](*)(C)C,-99.0000,-99.0000,0.1032,-99.0000
3714,*CCCCCCCCCCCCCCCC[N+](C)(C)CCCCCC[N+](*)(C)C,-99.0000,-99.0000,0.1771,-99.0000


In [3]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=1)

In [4]:
train_data.reset_index(inplace=True, drop=True)

In [5]:
test_data.reset_index(inplace=True, drop=True)
test_data

,smiles,Eea,Egb,Egc,Ei
0,*C(F)(F)C(F)(F)ON*,-99.0,-99.0000,6.8893,-99.0000
1,*c1ccc(-c2ccc(-c3cc(-c4ccccc4)c4cc5c(-c6ccccc6...,-99.0,-99.0000,2.8216,-99.0000
2,*C(C=C1)=CC=C1OC(C=C2)=CC=C2O*,-99.0,-99.0000,4.7875,-99.0000
3,*C=CC1CC(*)C(C(=O)Oc2ccccc2)C1C(=O)Oc1ccccc1,-99.0,-99.0000,5.2819,-99.0000
4,*CC(*)C(=O)OCCCCC,-99.0,-99.0000,6.4787,-99.0000
...,...,...,...,...,...
739,*CCC(S1)=CC=C1O*,-99.0,-99.0000,4.8828,-99.0000
740,*CCCCCCCCOC(=O)CCCCCCC(=O)O*,-99.0,-99.0000,6.7527,-99.0000
741,*NC(S1)=CC=C1C(=O)C(S2)=CC=C2*,-99.0,-99.0000,2.2308,-99.0000
742,*CC(O)COC(=O)CCCCC(=O)OCC(O)COc1ccc(O*)cc1,-99.0,-99.0000,5.0285,-99.0000


In [6]:
properties = ['Eea','Egb','Egc','Ei']
train_data_aug = train_data.copy()
train_data

,smiles,Eea,Egb,Egc,Ei
0,*CCOC(=O)c1ccc2ccc(C(=O)O*)cc2c1,-99.0000,-99.0000,3.9507,-99.000
1,*CCCCCNC(=O)CC(=O)N*,-99.0000,-99.0000,5.7523,-99.000
2,*CC(CBr)(CBr)CO*,-99.0000,-99.0000,5.9930,-99.000
3,*NC(S1)=CC=C1C(S2)=CC=C2C(=S)*,-99.0000,-99.0000,2.0192,-99.000
4,*CCCCCCCCCCc1nnc(*)n1N,-99.0000,-99.0000,5.6319,-99.000
...,...,...,...,...,...
2967,*c1ccc(C(C=CC)c2ccc(-n3c(=O)c4cc5c(=O)n(*)c(=O...,-99.0000,-99.0000,2.5577,-99.000
2968,*CCOCCOCCOC(=O)CC(=O)O*,-99.0000,-99.0000,5.6203,-99.000
2969,*CCCCCCCCCCCCOC(=O)CCCCCNC(=O)CCCCC(=O)NCCCCCC...,-99.0000,-99.0000,5.3312,-99.000
2970,*Nc1ccc(C(=O)C(*)=S)s1,4.1937,1.4251,-99.0000,6.311


In [8]:
DataAug = MultiTask_DataAugmentation(2)
train_data_aug = DataAug.smiles_augmentation(train_data_aug)
train_data_aug

,smiles,Eea,Egb,Egc,Ei
0,*CCOC(=O)c1ccc2ccc(C(=O)O*)cc2c1,-99.0,-99.0,3.9507,-99.0
1,C(OC(=O)c1ccc2ccc(C(=O)O*)cc2c1)C*,-99.0,-99.0,3.9507,-99.0
2,*CCCCCNC(=O)CC(=O)N*,-99.0,-99.0,5.7523,-99.0
3,C(NC(=O)CC(=O)N*)CCCC*,-99.0,-99.0,5.7523,-99.0
4,*CC(CBr)(CBr)CO*,-99.0,-99.0,5.993,-99.0
...,...,...,...,...,...
11698,*C(=S)C(c1sc(N*)cc1)=O,4.1937,1.4251,-99.0,6.311
11699,S=C(C(c1sc(N*)cc1)=O)*,4.1937,1.4251,-99.0,6.311
11700,s1c(N*)ccc1C(=O)C(*)=S,4.1937,1.4251,-99.0,6.311
11701,*CCCCCOC(=O)c1cccc(C(=O)O*)c1,-99.0,-99.0,5.0521,-99.0


In [9]:
scaler_aug = []
for i,property in enumerate(properties):
    scaled = StandardScaler()
    train_data_aug.loc[train_data_aug[property] != -99, property] = scaled.fit_transform(
                    train_data_aug.loc[train_data_aug[property] != -99, [property]]).flatten()
                
    scaler_aug.append(scaled)
train_data_aug

,smiles,Eea,Egb,Egc,Ei
0,*CCOC(=O)c1ccc2ccc(C(=O)O*)cc2c1,-99.0,-99.0,-0.547541,-99.0
1,C(OC(=O)c1ccc2ccc(C(=O)O*)cc2c1)C*,-99.0,-99.0,-0.547541,-99.0
2,*CCCCCNC(=O)CC(=O)N*,-99.0,-99.0,0.56452,-99.0
3,C(NC(=O)CC(=O)N*)CCCC*,-99.0,-99.0,0.56452,-99.0
4,*CC(CBr)(CBr)CO*,-99.0,-99.0,0.713095,-99.0
...,...,...,...,...,...
11698,*C(=S)C(c1sc(N*)cc1)=O,1.838817,-1.505222,-99.0,0.179108
11699,S=C(C(c1sc(N*)cc1)=O)*,1.838817,-1.505222,-99.0,0.179108
11700,s1c(N*)ccc1C(=O)C(*)=S,1.838817,-1.505222,-99.0,0.179108
11701,*CCCCCOC(=O)c1cccc(C(=O)O*)c1,-99.0,-99.0,0.132312,-99.0


In [10]:
for i,property in enumerate(properties):
    train_data_aug.loc[train_data_aug[property] != -99, property] = scaler_aug[i].inverse_transform(train_data_aug.loc[train_data_aug[property] != -99, [property]]).flatten()
train_data_aug

,smiles,Eea,Egb,Egc,Ei
0,*CCOC(=O)c1ccc2ccc(C(=O)O*)cc2c1,-99.0,-99.0,3.9507,-99.0
1,C(OC(=O)c1ccc2ccc(C(=O)O*)cc2c1)C*,-99.0,-99.0,3.9507,-99.0
2,*CCCCCNC(=O)CC(=O)N*,-99.0,-99.0,5.7523,-99.0
3,C(NC(=O)CC(=O)N*)CCCC*,-99.0,-99.0,5.7523,-99.0
4,*CC(CBr)(CBr)CO*,-99.0,-99.0,5.993,-99.0
...,...,...,...,...,...
11698,*C(=S)C(c1sc(N*)cc1)=O,4.1937,1.4251,-99.0,6.311
11699,S=C(C(c1sc(N*)cc1)=O)*,4.1937,1.4251,-99.0,6.311
11700,s1c(N*)ccc1C(=O)C(*)=S,4.1937,1.4251,-99.0,6.311
11701,*CCCCCOC(=O)c1cccc(C(=O)O*)c1,-99.0,-99.0,5.0521,-99.0
